# Secondary standard stars

In [20]:
import astropy
from astropy.io import fits
from astropy.stats import sigma_clipped_stats
from astropy.coordinates import SkyCoord
from astropy import units as u
import matplotlib.pyplot as plt
import numpy as np
import os
from photutils.aperture import CircularAperture, CircularAnnulus, aperture_photometry
from astropy.wcs import WCS
import warnings
from astropy.wcs import FITSFixedWarning
import pandas as pd

warnings.filterwarnings('ignore', category=FITSFixedWarning, message=".*'datfix' made the change.*")

# --- Configuration ---
wd = r"C:\Users\friesco\workstation\fr-p\studies\ASTRO716\data_excercise"  # Your working directory
plt.rcParams['figure.figsize'] = [10, 8]

# --- Functions ---
def phot_counts(img, nx, ny, ap_rad, in_ann, out_ann):
    # ... (same as before)
    ap = CircularAperture((nx, ny), r=ap_rad)
    ann_ap = CircularAnnulus((nx, ny), r_in=in_ann, r_out=out_ann)
    phot_tbl = aperture_photometry(img, [ap, ann_ap])

    ann_mask = ann_ap.to_mask(method='center')
    ann_data = ann_mask.multiply(img)
    bkg_vals = ann_data[ann_mask.data > 0]
    mean, med, std = sigma_clipped_stats(bkg_vals, sigma=3.0)
    bkg_med = med
    bkg_sum = bkg_med * ap.area

    tot_counts = phot_tbl['aperture_sum_0'][0]
    return tot_counts, bkg_sum, ap.area

def calc_snr(tot_counts, bkg_counts, ap_area):
    # ... (same as before)
    mean_counts = (tot_counts - bkg_counts) / ap_area
    if tot_counts + bkg_counts <= 0:
        print("Warning: Negative counts. Returning SNR = 0")
        return 0.0
    err_counts = np.sqrt(tot_counts + bkg_counts) / ap_area
    return mean_counts / err_counts

def phot_mag(img, x, y, ap_rad, in_ann, out_ann):
    # ... (same as before)
    ap = CircularAperture((x, y), r=ap_rad)
    ann_ap = CircularAnnulus((x, y), r_in=in_ann, r_out=out_ann)
    phot_tbl = aperture_photometry(img, [ap, ann_ap])

    ann_mask = ann_ap.to_mask(method='center')
    ann_data = ann_mask.multiply(img)
    bkg_vals = ann_data[ann_mask.data > 0]
    mean, med, std = sigma_clipped_stats(bkg_vals, sigma=3.0)
    bkg_med = med
    bkg_sum = bkg_med * ap.area

    final_sum = phot_tbl['aperture_sum_0'] - bkg_sum
    if final_sum[0] > 0:
        inst_mag = -2.5 * np.log10(final_sum) + 50
        mag_err = 1.0857 * (np.sqrt(phot_tbl['aperture_sum_0'] + bkg_sum) / phot_tbl['aperture_sum_0'])
    else:
        inst_mag = np.nan
        mag_err = np.nan
    return inst_mag, mag_err, bkg_med

# --- Load PS1 Catalog ---
ps1_cat = pd.read_csv(os.path.join(wd, 'psdr1_new.tsv'), sep='\t')

# --- Coordinates of Secondary Stars ---
# Only the coordinates
sec_stars = {
    'star1': {'coords': (820, 165)},
    'star2': {'coords': (1795, 520)},
    'star3': {'coords': (935, 565)},
    'star4': {'coords': (745, 1315)},
    'star5': {'coords': (935, 1020)}
}

# --- Photometry Parameters ---
ap_radii_snr = np.arange(5, 41, 1)
in_ann, out_ann = 25, 35

# --- Get Optimal Apertures for Each Image ---
phot_files = [f for f in os.listdir(wd) if f.startswith('phot_') and f.endswith('.fits')]
phot_files.sort()
opt_ap = []

# --- Placeholder for the nova coordinates ---
nx, ny = 1033, 1336

for i, f in enumerate(phot_files):
    path = os.path.join(wd, f)
    with fits.open(path) as hdu:
        img = hdu[0].data
        if img.dtype.byteorder == '>':
            img = img.byteswap().view(img.dtype.newbyteorder('<'))

    print(f"\nProcessing: {f}")
    snrs = []
    for ap_rad in ap_radii_snr:
        tot_counts, bkg_counts, ap_area = phot_counts(img, nx, ny, ap_rad, in_ann, out_ann)
        if tot_counts + bkg_counts <= 0:
            print(f"Warning: Negative counts in {f}, aperture: {ap_rad}")
            snrs.append(0.0)
        else:
            snr = calc_snr(tot_counts, bkg_counts, ap_area)
            snrs.append(snr)

    opt_ap_rad = ap_radii_snr[np.argmax(snrs)]
    opt_ap.append(opt_ap_rad)
    print(f"  Optimal Aperture: {opt_ap_rad} px")

# --- Perform Photometry and Calibration ---
calibration_results = []

for i, f in enumerate(phot_files):
    path = os.path.join(wd, f)
    with fits.open(path) as hdu:
        img = hdu[0].data
        hdr = hdu[0].header
        if img.dtype.byteorder == '>':
            img = img.byteswap().view(img.dtype.newbyteorder('<'))
    wcs = WCS(hdr)

    print(f"\nCalibrating: {f}")
    opt_ap_rad = opt_ap[i]
    image_results = {'image': f, 'mjd': hdr['MJD'], 'sec_stars': {}}

    # --- Photometry and Calibration for Secondary Stars ---
    zp_values = []
    for star_name, star_data in sec_stars.items():
        x, y = star_data['coords']
        inst_mag, mag_err, bkg_med = phot_mag(img, x, y, opt_ap_rad, in_ann, out_ann)
        ra, dec = wcs.pixel_to_world_values(x, y)

        # --- Create SkyCoord object ---
        c = SkyCoord(ra=ra, dec=dec, unit=u.deg)

        # --- Potential RA/DEC column names ---
        ra_col_names = ["RA", "RAJ2000", "raMean", "ra_mean"]
        dec_col_names = ["DEC", "DEJ2000", "decMean", "dec_mean"]

        # --- Find PS1 Catalog Magnitude ---
        best_match = None
        min_dist = float('inf')

        for ra_col in ra_col_names:
            for dec_col in dec_col_names:
                try:
                    # Convert to numeric
                    ps1_cat[ra_col] = pd.to_numeric(ps1_cat[ra_col], errors='coerce')
                    ps1_cat[dec_col] = pd.to_numeric(ps1_cat[dec_col], errors='coerce')

                    # Calculate distances
                    distances = np.sqrt(
                        (ps1_cat[ra_col] - c.ra.deg) ** 2 + (ps1_cat[dec_col] - c.dec.deg) ** 2
                    )

                    # Find closest match within search radius
                    closest_match_index = np.argmin(distances)
                    dist = distances[closest_match_index]

                    if dist < min_dist and dist < 0.0083:  # 30 arcseconds in degrees
                        min_dist = dist
                        best_match = ps1_cat.iloc[closest_match_index]

                except (KeyError, ValueError):
                    pass

        # --- Process best match ---
        if best_match is not None:
            # Extract r and g magnitudes
            ps1_r = best_match.get('rMeanPSFMag', np.nan)
            ps1_g = best_match.get('gMeanPSFMag', np.nan)
        else:
            print(f"Warning: No PS1 match found for {star_name} in {f}")
            ps1_r, ps1_g = np.nan, np.nan

        # --- Transform PS1 Magnitude to Sloan ---
        if not np.isnan(ps1_r) and not np.isnan(ps1_g):
            sloan_r = ps1_r - 0.001 + 0.011 * (ps1_g - ps1_r)
        else:
            sloan_r = np.nan

        # --- Calculate Magnitude Difference and Zero Point ---
        if not np.isnan(inst_mag[0]) and not np.isnan(sloan_r):
            delta_mag = sloan_r - inst_mag[0]
            zp_values.append(delta_mag)
        else:
            delta_mag = np.nan

        image_results['sec_stars'][star_name] = {
            'x': x,
            'y': y,
            'ra': ra,
            'dec': dec,
            'inst_mag': inst_mag[0],
            'mag_err': mag_err[0],
            'bkg': bkg_med,
            'ps1_r': ps1_r,
            'ps1_g': ps1_g,
            'sloan_r': sloan_r,
            'delta_mag': delta_mag
        }

    # --- Calculate Average Zero Point for This Image ---
    if zp_values:
        zp = np.nanmean(zp_values)
        zp_std = np.nanstd(zp_values)
    else:
        zp = np.nan
        zp_std = np.nan

    image_results['zp'] = zp
    image_results['zp_std'] = zp_std

    calibration_results.append(image_results)

    # --- Print Calibration Summary ---
    print("  Calibration Summary:")
    for star_name, star_data in image_results['sec_stars'].items():
        print(f"    {star_name}:")
        print(f"      Instrumental Mag: {star_data['inst_mag']:.3f} +/- {star_data['mag_err']:.3f}")
        print(f"      PS1 r: {star_data['ps1_r']:.3f}" if not np.isnan(star_data['ps1_r']) else "      PS1 r: N/A")
        print(f"      PS1 g: {star_data['ps1_g']:.3f}" if not np.isnan(star_data['ps1_g']) else "      PS1 g: N/A")
        print(f"      Sloan r': {star_data['sloan_r']:.3f}" if not np.isnan(star_data['sloan_r']) else "      Sloan r': N/A")
        print(f"      Delta Mag: {star_data['delta_mag']:.3f}" if not np.isnan(star_data['delta_mag']) else "      Delta Mag: N/A")

    print(f"  Zero Point (ZP): {zp:.3f} +/- {zp_std:.3f}" if not np.isnan(zp) else "  Zero Point (ZP): N/A +/- N/A")

# --- Create a DataFrame for Easier Analysis ---
df = pd.DataFrame(calibration_results)

# --- Save Calibration Results to CSV ---
output_file = os.path.join(wd, 'calibration_results.csv')
flat_calibration_results = []
for r in calibration_results:
    flat_row = {'image': r['image'], 'mjd': r['mjd'], 'zp': r['zp'], 'zp_std': r['zp_std']}
    for star_name, star_data in r['sec_stars'].items():
        flat_row[f'{star_name}_x'] = star_data['x']
        flat_row[f'{star_name}_y'] = star_data['y']
        flat_row[f'{star_name}_ra'] = star_data['ra']
        flat_row[f'{star_name}_dec'] = star_data['dec']
        flat_row[f'{star_name}_inst_mag'] = star_data['inst_mag']
        flat_row[f'{star_name}_mag_err'] = star_data['mag_err']
        flat_row[f'{star_name}_bkg'] = star_data['bkg']
        flat_row[f'{star_name}_ps1_r'] = star_data['ps1_r']
        flat_row[f'{star_name}_ps1_g'] = star_data['ps1_g']
        flat_row[f'{star_name}_sloan_r'] = star_data['sloan_r']
        flat_row[f'{star_name}_delta_mag'] = star_data['delta_mag']
    flat_calibration_results.append(flat_row)

df_flat = pd.DataFrame(flat_calibration_results)
df_flat.to_csv(output_file, index=False)
print(f"Calibration results saved to {output_file}")


Processing: phot_00.fits
  Optimal Aperture: 25 px

Processing: phot_01.fits
  Optimal Aperture: 27 px

Processing: phot_02.fits
  Optimal Aperture: 26 px

Processing: phot_03.fits
  Optimal Aperture: 26 px

Processing: phot_04.fits
  Optimal Aperture: 27 px

Processing: phot_05.fits
  Optimal Aperture: 25 px

Processing: phot_06.fits
  Optimal Aperture: 26 px

Processing: phot_07.fits
  Optimal Aperture: 30 px

Processing: phot_08.fits
  Optimal Aperture: 33 px

Processing: phot_09.fits
  Optimal Aperture: 29 px

Processing: phot_10.fits
  Optimal Aperture: 29 px

Processing: phot_11.fits
  Optimal Aperture: 30 px

Processing: phot_12.fits
  Optimal Aperture: 30 px

Processing: phot_13.fits
  Optimal Aperture: 29 px

Processing: phot_14.fits
  Optimal Aperture: 29 px

Processing: phot_15.fits
  Optimal Aperture: 28 px

Processing: phot_16.fits
  Optimal Aperture: 16 px

Processing: phot_17.fits
  Optimal Aperture: 17 px

Calibrating: phot_00.fits
  Calibration Summary:
    star1:
   

In [21]:
import astropy
from astropy.io import fits
from astropy.stats import sigma_clipped_stats
from astropy.coordinates import SkyCoord
from astropy import units as u
import matplotlib.pyplot as plt
import numpy as np
import os
from photutils.aperture import CircularAperture, CircularAnnulus, aperture_photometry
from astropy.wcs import WCS
import warnings
from astropy.wcs import FITSFixedWarning
import pandas as pd

warnings.filterwarnings('ignore', category=FITSFixedWarning, message=".*'datfix' made the change.*")

# --- Configuration ---
wd = r"C:\Users\friesco\workstation\fr-p\studies\ASTRO716\data_excercise"  # Your working directory
plt.rcParams['figure.figsize'] = [10, 8]

# --- Functions ---
def phot_counts(img, nx, ny, ap_rad, in_ann, out_ann):
    ap = CircularAperture((nx, ny), r=ap_rad)
    ann_ap = CircularAnnulus((nx, ny), r_in=in_ann, r_out=out_ann)
    phot_tbl = aperture_photometry(img, [ap, ann_ap])

    ann_mask = ann_ap.to_mask(method='center')
    ann_data = ann_mask.multiply(img)
    bkg_vals = ann_data[ann_mask.data > 0]
    mean, med, std = sigma_clipped_stats(bkg_vals, sigma=3.0)
    bkg_med = med
    bkg_sum = bkg_med * ap.area

    tot_counts = phot_tbl['aperture_sum_0'][0]
    return tot_counts, bkg_sum, ap.area

def calc_snr(tot_counts, bkg_counts, ap_area):
    mean_counts = (tot_counts - bkg_counts) / ap_area
    if tot_counts + bkg_counts <= 0:
        print("Warning: Negative counts. Returning SNR = 0")
        return 0.0
    err_counts = np.sqrt(tot_counts + bkg_counts) / ap_area
    return mean_counts / err_counts

def phot_mag(img, x, y, ap_rad, in_ann, out_ann):
    ap = CircularAperture((x, y), r=ap_rad)
    ann_ap = CircularAnnulus((x, y), r_in=in_ann, r_out=out_ann)
    phot_tbl = aperture_photometry(img, [ap, ann_ap])

    ann_mask = ann_ap.to_mask(method='center')
    ann_data = ann_mask.multiply(img)
    bkg_vals = ann_data[ann_mask.data > 0]
    mean, med, std = sigma_clipped_stats(bkg_vals, sigma=3.0)
    bkg_med = med
    bkg_sum = bkg_med * ap.area

    final_sum = phot_tbl['aperture_sum_0'] - bkg_sum
    if final_sum[0] > 0:
        inst_mag = -2.5 * np.log10(final_sum) + 50
        mag_err = 1.0857 * (np.sqrt(phot_tbl['aperture_sum_0'] + bkg_sum) / phot_tbl['aperture_sum_0'])
    else:
        inst_mag = np.nan
        mag_err = np.nan
    return inst_mag, mag_err, bkg_med

# --- Load PS1 Catalog ---
ps1_cat = pd.read_csv(os.path.join(wd, 'psdr1_new.tsv'), sep='\t')

# --- Coordinates of Secondary Stars ---
sec_stars = {
    'star1': {'coords': (820, 165)},
    'star2': {'coords': (1795, 520)},
    'star3': {'coords': (935, 565)},
    'star4': {'coords': (745, 1315)},
    'star5': {'coords': (935, 1020)}
}

# --- Photometry Parameters ---
in_ann, out_ann = 25, 35

# --- Get Optimal Apertures for Each Image ---
phot_files = [f for f in os.listdir(wd) if f.startswith('phot_') and f.endswith('.fits')]
phot_files.sort()
opt_ap = []

# --- Placeholder for the nova coordinates ---
nx, ny = 1033, 1336

for i, f in enumerate(phot_files):
    path = os.path.join(wd, f)
    with fits.open(path) as hdu:
        img = hdu[0].data
        if img.dtype.byteorder == '>':
            img = img.byteswap().view(img.dtype.newbyteorder('<'))

    print(f"\nProcessing: {f}")
    snrs = []
    for ap_rad in ap_radii_snr:
        tot_counts, bkg_counts, ap_area = phot_counts(img, nx, ny, ap_rad, in_ann, out_ann)
        if tot_counts + bkg_counts <= 0:
            print(f"Warning: Negative counts in {f}, aperture: {ap_rad}")
            snrs.append(0.0)
        else:
            snr = calc_snr(tot_counts, bkg_counts, ap_area)
            snrs.append(snr)

    opt_ap_rad = ap_radii_snr[np.argmax(snrs)]
    opt_ap.append(opt_ap_rad)
    print(f"  Optimal Aperture: {opt_ap_rad} px")

# --- Perform Photometry and Calibration ---
calibration_results = []

for i, f in enumerate(phot_files):
    path = os.path.join(wd, f)
    with fits.open(path) as hdu:
        img = hdu[0].data
        hdr = hdu[0].header
        if img.dtype.byteorder == '>':
            img = img.byteswap().view(img.dtype.newbyteorder('<'))
    wcs = WCS(hdr)

    print(f"\nCalibrating: {f}")
    opt_ap_rad = opt_ap[i]
    image_results = {'image': f, 'mjd': hdr['MJD'], 'sec_stars': {}}

    # --- Photometry and Calibration for Secondary Stars ---
    zp_values = []
    for star_name, star_data in sec_stars.items():
        x, y = star_data['coords']
        inst_mag, mag_err, bkg_med = phot_mag(img, x, y, opt_ap_rad, in_ann, out_ann)
        ra, dec = wcs.pixel_to_world_values(x, y)

        # --- Create SkyCoord object ---
        c = SkyCoord(ra=ra, dec=dec, unit=u.deg)

        # --- Potential RA/DEC column names ---
        ra_col_names = ["RA", "RAJ2000", "raMean", "ra_mean"]
        dec_col_names = ["DEC", "DEJ2000", "decMean", "dec_mean"]

        # --- Find PS1 Catalog Magnitude ---
        best_match = None
        min_dist = float('inf')

        for ra_col in ra_col_names:
            for dec_col in dec_col_names:
                try:
                    # Convert to numeric
                    ps1_cat[ra_col] = pd.to_numeric(ps1_cat[ra_col], errors='coerce')
                    ps1_cat[dec_col] = pd.to_numeric(ps1_cat[dec_col], errors='coerce')

                    # Calculate distances
                    distances = np.sqrt(
                        (ps1_cat[ra_col] - c.ra.deg) ** 2 + (ps1_cat[dec_col] - c.dec.deg) ** 2
                    )

                    # Find closest match within search radius
                    closest_match_index = np.argmin(distances)
                    dist = distances[closest_match_index]

                    if dist < min_dist and dist < 0.0083:  # 30 arcseconds in degrees
                        min_dist = dist
                        best_match = ps1_cat.iloc[closest_match_index]

                except (KeyError, ValueError):
                    pass

        # --- Process best match ---
        if best_match is not None:
            # Extract r and g magnitudes
            ps1_r = best_match.get('rMeanPSFMag', np.nan)
            ps1_g = best_match.get('gMeanPSFMag', np.nan)
        else:
            print(f"Warning: No PS1 match found for {star_name} in {f}")
            ps1_r, ps1_g = np.nan, np.nan

        # --- Transform PS1 Magnitude to Sloan ---
        if not np.isnan(ps1_r) and not np.isnan(ps1_g):
            sloan_r = ps1_r - 0.001 + 0.011 * (ps1_g - ps1_r)
        else:
            sloan_r = np.nan

        # --- Calculate Magnitude Difference and Zero Point ---
        if not np.isnan(inst_mag[0]) and not np.isnan(sloan_r):
            delta_mag = sloan_r - inst_mag[0]
            zp_values.append(delta_mag)
        else:
            delta_mag = np.nan

        image_results['sec_stars'][star_name] = {
            'x': x,
            'y': y,
            'ra': ra,
            'dec': dec,
            'inst_mag': inst_mag[0],
            'mag_err': mag_err[0],
            'bkg': bkg_med,
            'ps1_r': ps1_r,
            'ps1_g': ps1_g,
            'sloan_r': sloan_r,
            'delta_mag': delta_mag
        }

    # --- Calculate Average Zero Point for This Image ---
    if zp_values:
        zp = np.nanmean(zp_values)
        zp_std = np.nanstd(zp_values)
    else:
        zp = np.nan
        zp_std = np.nan

    image_results['zp'] = zp
    image_results['zp_std'] = zp_std

    calibration_results.append(image_results)

    # --- Print Calibration Summary ---
    print("  Calibration Summary:")
    for star_name, star_data in image_results['sec_stars'].items():
        print(f"    {star_name}:")
        print(f"      Instrumental Mag: {star_data['inst_mag']:.3f} +/- {star_data['mag_err']:.3f}")
        print(f"      PS1 r: {star_data['ps1_r']:.3f}" if not np.isnan(star_data['ps1_r']) else "      PS1 r: N/A")
        print(f"      PS1 g: {star_data['ps1_g']:.3f}" if not np.isnan(star_data['ps1_g']) else "      PS1 g: N/A")
        print(f"      Sloan r': {star_data['sloan_r']:.3f}" if not np.isnan(star_data['sloan_r']) else "      Sloan r': N/A")
        print(f"      Delta Mag: {star_data['delta_mag']:.3f}" if not np.isnan(star_data['delta_mag']) else "      Delta Mag: N/A")

    print(f"  Zero Point (ZP): {zp:.3f} +/- {zp_std:.3f}" if not np.isnan(zp) else "  Zero Point (ZP): N/A +/- N/A")

# --- Create a DataFrame for Easier Analysis ---
df = pd.DataFrame(calibration_results)

# --- Save Calibration Results to CSV ---
output_file = os.path.join(wd, 'calibration_results.csv')
flat_calibration_results = []
for r in calibration_results:
    flat_row = {'image': r['image'], 'mjd': r['mjd'], 'zp': r['zp'], 'zp_std': r['zp_std']}
    for star_name, star_data in r['sec_stars'].items():
        flat_row[f'{star_name}_x'] = star_data['x']
        flat_row[f'{star_name}_y'] = star_data['y']
        flat_row[f'{star_name}_ra'] = star_data['ra']
        flat_row[f'{star_name}_dec'] = star_data['dec']
        flat_row[f'{star_name}_inst_mag'] = star_data['inst_mag']
        flat_row[f'{star_name}_mag_err'] = star_data['mag_err']
        flat_row[f'{star_name}_bkg'] = star_data['bkg']
        flat_row[f'{star_name}_ps1_r'] = star_data['ps1_r']
        flat_row[f'{star_name}_ps1_g'] = star_data['ps1_g']
        flat_row[f'{star_name}_sloan_r'] = star_data['sloan_r']
        flat_row[f'{star_name}_delta_mag'] = star_data['delta_mag']
    flat_calibration_results.append(flat_row)

df_flat = pd.DataFrame(flat_calibration_results)
df_flat.to_csv(output_file, index=False)
print(f"Calibration results saved to {output_file}")


Processing: phot_00.fits
  Optimal Aperture: 25 px

Processing: phot_01.fits
  Optimal Aperture: 27 px

Processing: phot_02.fits
  Optimal Aperture: 26 px

Processing: phot_03.fits
  Optimal Aperture: 26 px

Processing: phot_04.fits
  Optimal Aperture: 27 px

Processing: phot_05.fits
  Optimal Aperture: 25 px

Processing: phot_06.fits
  Optimal Aperture: 26 px

Processing: phot_07.fits
  Optimal Aperture: 30 px

Processing: phot_08.fits
  Optimal Aperture: 33 px

Processing: phot_09.fits
  Optimal Aperture: 29 px

Processing: phot_10.fits
  Optimal Aperture: 29 px

Processing: phot_11.fits
  Optimal Aperture: 30 px

Processing: phot_12.fits
  Optimal Aperture: 30 px

Processing: phot_13.fits
  Optimal Aperture: 29 px

Processing: phot_14.fits
  Optimal Aperture: 29 px

Processing: phot_15.fits
  Optimal Aperture: 28 px

Processing: phot_16.fits
  Optimal Aperture: 16 px

Processing: phot_17.fits
  Optimal Aperture: 17 px

Calibrating: phot_00.fits
  Calibration Summary:
    star1:
   

In [22]:
import astropy
from astropy.io import fits
from astropy.stats import sigma_clipped_stats
from astropy.coordinates import SkyCoord
from astropy import units as u
import matplotlib.pyplot as plt
import numpy as np
import os
from photutils.aperture import CircularAperture, CircularAnnulus, aperture_photometry
from astropy.wcs import WCS
import warnings
from astropy.wcs import FITSFixedWarning
import pandas as pd

warnings.filterwarnings('ignore', category=FITSFixedWarning, message=".*'datfix' made the change.*")

# --- Configuration ---
wd = r"C:\Users\friesco\workstation\fr-p\studies\ASTRO716\data_excercise"  # Your working directory
plt.rcParams['figure.figsize'] = [10, 8]

# --- Functions ---
def phot_counts(img, nx, ny, ap_rad, in_ann, out_ann):
    ap = CircularAperture((nx, ny), r=ap_rad)
    ann_ap = CircularAnnulus((nx, ny), r_in=in_ann, r_out=out_ann)
    phot_tbl = aperture_photometry(img, [ap, ann_ap])

    ann_mask = ann_ap.to_mask(method='center')
    ann_data = ann_mask.multiply(img)
    bkg_vals = ann_data[ann_mask.data > 0]
    mean, med, std = sigma_clipped_stats(bkg_vals, sigma=3.0)
    bkg_med = med
    bkg_sum = bkg_med * ap.area

    tot_counts = phot_tbl['aperture_sum_0'][0]
    return tot_counts, bkg_sum, ap.area

def calc_snr(tot_counts, bkg_counts, ap_area):
    mean_counts = (tot_counts - bkg_counts) / ap_area
    if tot_counts + bkg_counts <= 0:
        print("Warning: Negative counts. Returning SNR = 0")
        return 0.0
    err_counts = np.sqrt(tot_counts + bkg_counts) / ap_area
    return mean_counts / err_counts

def phot_mag(img, x, y, ap_rad, in_ann, out_ann):
    ap = CircularAperture((x, y), r=ap_rad)
    ann_ap = CircularAnnulus((x, y), r_in=in_ann, r_out=out_ann)
    phot_tbl = aperture_photometry(img, [ap, ann_ap])

    ann_mask = ann_ap.to_mask(method='center')
    ann_data = ann_mask.multiply(img)
    bkg_vals = ann_data[ann_mask.data > 0]
    mean, med, std = sigma_clipped_stats(bkg_vals, sigma=3.0)
    bkg_med = med
    bkg_sum = bkg_med * ap.area

    final_sum = phot_tbl['aperture_sum_0'] - bkg_sum
    if final_sum[0] > 0:
        inst_mag = -2.5 * np.log10(final_sum) + 50
        mag_err = 1.0857 * (np.sqrt(phot_tbl['aperture_sum_0'] + bkg_sum) / phot_tbl['aperture_sum_0'])
    else:
        inst_mag = np.nan
        mag_err = np.nan
    return inst_mag, mag_err, bkg_med

# --- Load Calibration Data ---
calibration_file = os.path.join(wd, 'calibration_results.csv')
calibration_df = pd.read_csv(calibration_file)

# --- Photometry Parameters ---
in_ann, out_ann = 25, 35

# --- Nova Coordinates ---
nx, ny = 1033, 1336

# --- Get Optimal Apertures for Each Image ---
phot_files = [f for f in os.listdir(wd) if f.startswith('phot_') and f.endswith('.fits')]
phot_files.sort()
opt_ap = []
ap_radii_snr = np.arange(5, 41, 1)
for i, f in enumerate(phot_files):
    path = os.path.join(wd, f)
    with fits.open(path) as hdu:
        img = hdu[0].data
        if img.dtype.byteorder == '>':
            img = img.byteswap().view(img.dtype.newbyteorder('<'))

    print(f"\nProcessing: {f}")
    snrs = []
    for ap_rad in ap_radii_snr:
        tot_counts, bkg_counts, ap_area = phot_counts(img, nx, ny, ap_rad, in_ann, out_ann)
        if tot_counts + bkg_counts <= 0:
            print(f"Warning: Negative counts in {f}, aperture: {ap_rad}")
            snrs.append(0.0)
        else:
            snr = calc_snr(tot_counts, bkg_counts, ap_area)
            snrs.append(snr)

    opt_ap_rad = ap_radii_snr[np.argmax(snrs)]
    opt_ap.append(opt_ap_rad)
    print(f"  Optimal Aperture: {opt_ap_rad} px")

# --- Perform Photometry on Nova and Apply Calibration ---
nova_phot_results = []

for i, f in enumerate(phot_files):
    path = os.path.join(wd, f)
    with fits.open(path) as hdu:
        img = hdu[0].data
        hdr = hdu[0].header
        if img.dtype.byteorder == '>':
            img = img.byteswap().view(img.dtype.newbyteorder('<'))
    wcs = WCS(hdr)

    print(f"\nApplying calibration to: {f}")
    opt_ap_rad = opt_ap[i]

    # --- Find Calibration Data for This Image ---
    image_calib_data = calibration_df[calibration_df['image'] == f].iloc[0]
    zp = image_calib_data['zp']
    zp_std = image_calib_data['zp_std']

    # --- Photometry for Nova ---
    inst_mag, mag_err, bkg_med = phot_mag(img, nx, ny, opt_ap_rad, in_ann, out_ann)
    ra, dec = wcs.pixel_to_world_values(nx, ny)

    # --- Apply Calibration ---
    if not np.isnan(inst_mag) and not np.isnan(zp):
        calibrated_mag = inst_mag + zp
        calibrated_mag_err = np.sqrt(mag_err**2 + zp_std**2)  # Propagate errors
    else:
        calibrated_mag = np.nan
        calibrated_mag_err = np.nan

    nova_phot_results.append({
        'image': f,
        'mjd': hdr['MJD'],
        'x': nx,
        'y': ny,
        'ra': ra,
        'dec': dec,
        'inst_mag': inst_mag,
        'mag_err': mag_err,
        'bkg': bkg_med,
        'zp': zp,
        'zp_std': zp_std,
        'calibrated_mag': calibrated_mag,
        'calibrated_mag_err': calibrated_mag_err
    })

    print(f"  Nova Instrumental Mag: {inst_mag:.3f} +/- {mag_err:.3f}")
    print(f"  Zero Point (ZP): {zp:.3f} +/- {zp_std:.3f}")
    print(f"  Nova Calibrated Mag: {calibrated_mag:.3f} +/- {calibrated_mag_err:.3f}")

# --- Create a DataFrame for Nova Photometry ---
nova_df = pd.DataFrame(nova_phot_results)

# --- Light Curve Plot ---
plt.figure()
plt.errorbar(nova_df['mjd'], nova_df['calibrated_mag'], yerr=nova_df['calibrated_mag_err'], fmt='bo-', capsize=4)
plt.xlabel("MJD")
plt.ylabel("Calibrated Magnitude (Sloan r')")
plt.title("Nova Light Curve")
plt.gca().invert_yaxis()
plt.grid(True)
plt.show()

# --- Save Nova Photometry Results to CSV ---
output_file = os.path.join(wd, 'nova_photometry_results.csv')
nova_df.to_csv(output_file, index=False)
print(f"Nova photometry results saved to {output_file}")


Processing: phot_00.fits
  Optimal Aperture: 25 px

Processing: phot_01.fits
  Optimal Aperture: 27 px

Processing: phot_02.fits
  Optimal Aperture: 26 px

Processing: phot_03.fits
  Optimal Aperture: 26 px

Processing: phot_04.fits
  Optimal Aperture: 27 px

Processing: phot_05.fits
  Optimal Aperture: 25 px

Processing: phot_06.fits
  Optimal Aperture: 26 px

Processing: phot_07.fits
  Optimal Aperture: 30 px

Processing: phot_08.fits
  Optimal Aperture: 33 px

Processing: phot_09.fits
  Optimal Aperture: 29 px

Processing: phot_10.fits
  Optimal Aperture: 29 px

Processing: phot_11.fits
  Optimal Aperture: 30 px

Processing: phot_12.fits
  Optimal Aperture: 30 px

Processing: phot_13.fits
  Optimal Aperture: 29 px

Processing: phot_14.fits
  Optimal Aperture: 29 px

Processing: phot_15.fits
  Optimal Aperture: 28 px

Processing: phot_16.fits
  Optimal Aperture: 16 px

Processing: phot_17.fits
  Optimal Aperture: 17 px

Applying calibration to: phot_00.fits


TypeError: unsupported format string passed to Column.__format__

In [23]:
import astropy
from astropy.io import fits
from astropy.stats import sigma_clipped_stats
from astropy.coordinates import SkyCoord
from astropy import units as u
import matplotlib.pyplot as plt
import numpy as np
import os
from photutils.aperture import CircularAperture, CircularAnnulus, aperture_photometry
from astropy.wcs import WCS
import warnings
from astropy.wcs import FITSFixedWarning
import pandas as pd

warnings.filterwarnings('ignore', category=FITSFixedWarning, message=".*'datfix' made the change.*")

# --- Configuration ---
wd = r"C:\Users\friesco\workstation\fr-p\studies\ASTRO716\data_excercise"  # Your working directory
plt.rcParams['figure.figsize'] = [10, 8]

# --- Functions ---
def phot_counts(img, nx, ny, ap_rad, in_ann, out_ann):
    ap = CircularAperture((nx, ny), r=ap_rad)
    ann_ap = CircularAnnulus((nx, ny), r_in=in_ann, r_out=out_ann)
    phot_tbl = aperture_photometry(img, [ap, ann_ap])

    ann_mask = ann_ap.to_mask(method='center')
    ann_data = ann_mask.multiply(img)
    bkg_vals = ann_data[ann_mask.data > 0]
    mean, med, std = sigma_clipped_stats(bkg_vals, sigma=3.0)
    bkg_med = med
    bkg_sum = bkg_med * ap.area

    tot_counts = phot_tbl['aperture_sum_0'][0]
    return tot_counts, bkg_sum, ap.area

def calc_snr(tot_counts, bkg_counts, ap_area):
    mean_counts = (tot_counts - bkg_counts) / ap_area
    if tot_counts + bkg_counts <= 0:
        print("Warning: Negative counts. Returning SNR = 0")
        return 0.0
    err_counts = np.sqrt(tot_counts + bkg_counts) / ap_area
    return mean_counts / err_counts

def phot_mag(img, x, y, ap_rad, in_ann, out_ann):
    ap = CircularAperture((x, y), r=ap_rad)
    ann_ap = CircularAnnulus((x, y), r_in=in_ann, r_out=out_ann)
    phot_tbl = aperture_photometry(img, [ap, ann_ap])

    ann_mask = ann_ap.to_mask(method='center')
    ann_data = ann_mask.multiply(img)
    bkg_vals = ann_data[ann_mask.data > 0]
    mean, med, std = sigma_clipped_stats(bkg_vals, sigma=3.0)
    bkg_med = med
    bkg_sum = bkg_med * ap.area

    final_sum = phot_tbl['aperture_sum_0'] - bkg_sum
    if final_sum[0] > 0:
        inst_mag = -2.5 * np.log10(final_sum) + 50
        mag_err = 1.0857 * (np.sqrt(phot_tbl['aperture_sum_0'] + bkg_sum) / phot_tbl['aperture_sum_0'])
    else:
        inst_mag = np.nan
        mag_err = np.nan
    return inst_mag, mag_err, bkg_med

# --- Load Calibration Data ---
calibration_file = os.path.join(wd, 'calibration_results.csv')
calibration_df = pd.read_csv(calibration_file)

# --- Photometry Parameters ---
in_ann, out_ann = 25, 35

# --- Load Validated Secondary Stars Data ---
# Make sure you have run the updated star identification code (previous response)
# to generate the validated_sec_stars dictionary
# For example:
validated_sec_stars = {
    'star1': {'gaia_coords': SkyCoord(ra=166.22191667*u.deg, dec=40.123575*u.deg, frame='icrs'), 'pixel_coords': (819.5418875703874, 164.45447973955848), 'ps1_r': 17.345, 'ps1_g': 18.186, 'sloan_r': 17.335779},
    'star2': {'gaia_coords': SkyCoord(ra=166.80870833*u.deg, dec=40.20586111*u.deg, frame='icrs'), 'pixel_coords': (1795.4067531293238, 519.961987653702), 'ps1_r': 16.758, 'ps1_g': 17.578, 'sloan_r': 16.75098},
    'star3': {'gaia_coords': SkyCoord(ra=166.883625*u.deg, dec=40.13329722*u.deg, frame='icrs'), 'pixel_coords': (935.4963748145617, 564.8516154108505), 'ps1_r': 16.707, 'ps1_g': 17.355, 'sloan_r': 16.701868},
    'star4': {'gaia_coords': SkyCoord(ra=168.121125*u.deg, dec=40.11721944*u.deg, frame='icrs'), 'pixel_coords': (745.4707890528338, 1314.5570097305657), 'ps1_r': 17.974, 'ps1_g': 19.047, 'sloan_r': 17.962196999999998},
    'star5': {'gaia_coords': SkyCoord(ra=167.63645833*u.deg, dec=40.13327222*u.deg, frame='icrs'), 'pixel_coords': (935.4584293955837, 1019.8253317574671), 'ps1_r': 18.061, 'ps1_g': 18.667, 'sloan_r': 18.054334}
}
# --- Nova's Pixel Coordinates (from your original code) ---
# Make sure these are the correct coordinates for your nova
nx, ny = 1033, 1336

# --- Get Optimal Apertures for Each Image ---
phot_files = [f for f in os.listdir(wd) if f.startswith('phot_') and f.endswith('.fits')]
phot_files.sort()
opt_ap = []
ap_radii_snr = np.arange(5, 41, 1)
for i, f in enumerate(phot_files):
    path = os.path.join(wd, f)
    with fits.open(path) as hdu:
        img = hdu[0].data
        if img.dtype.byteorder == '>':
            img = img.byteswap().view(img.dtype.newbyteorder('<'))

    print(f"\nProcessing: {f}")
    snrs = []
    for ap_rad in ap_radii_snr:
        tot_counts, bkg_counts, ap_area = phot_counts(img, nx, ny, ap_rad, in_ann, out_ann)
        if tot_counts + bkg_counts <= 0:
            print(f"Warning: Negative counts in {f}, aperture: {ap_rad}")
            snrs.append(0.0)
        else:
            snr = calc_snr(tot_counts, bkg_counts, ap_area)
            snrs.append(snr)

    opt_ap_rad = ap_radii_snr[np.argmax(snrs)]
    opt_ap.append(opt_ap_rad)
    print(f"  Optimal Aperture: {opt_ap_rad} px")

# --- Perform Photometry on Nova and Apply Calibration ---
nova_phot_results = []

for i, f in enumerate(phot_files):
    path = os.path.join(wd, f)
    with fits.open(path) as hdu:
        img = hdu[0].data
        hdr = hdu[0].header
        if img.dtype.byteorder == '>':
            img = img.byteswap().view(img.dtype.newbyteorder('<'))
    wcs = WCS(hdr)

    print(f"\nApplying calibration to: {f}")
    opt_ap_rad = opt_ap[i]

    # --- Find Calibration Data for This Image ---
    image_calib_data = calibration_df[calibration_df['image'] == f].iloc[0]
    zp = image_calib_data['zp']
    zp_std = image_calib_data['zp_std']

    # --- Photometry for Nova ---
    # Use the coordinates from validated_sec_stars
    inst_mag_col, mag_err_col, bkg_med = phot_mag(img, nx, ny, opt_ap_rad, in_ann, out_ann)

    # --- Access the float values from the Column objects ---
    inst_mag = inst_mag_col.item()
    mag_err = mag_err_col.item()

    ra, dec = wcs.pixel_to_world_values(nx, ny)

    # --- Apply Calibration ---
    if not np.isnan(inst_mag) and not np.isnan(zp):
        calibrated_mag = inst_mag + zp
        calibrated_mag_err = np.sqrt(mag_err**2 + zp_std**2)  # Propagate errors
    else:
        calibrated_mag = np.nan
        calibrated_mag_err = np.nan

    nova_phot_results.append({
        'image': f,
        'mjd': hdr['MJD'],
        'x': nx,
        'y': ny,
        'ra': ra,
        'dec': dec,
        'inst_mag': inst_mag,
        'mag_err': mag_err,
        'bkg': bkg_med,
        'zp': zp,
        'zp_std': zp_std,
        'calibrated_mag': calibrated_mag,
        'calibrated_mag_err': calibrated_mag_err
    })

    print(f"  Nova Instrumental Mag: {inst_mag:.3f} +/- {mag_err:.3f}")
    print(f"  Zero Point (ZP): {zp:.3f} +/- {zp_std:.3f}")
    print(f"  Nova Calibrated Mag: {calibrated_mag:.3f} +/- {calibrated_mag_err:.3f}")

# --- Create a DataFrame for Nova Photometry ---
nova_df = pd.DataFrame(nova_phot_results)

# --- Light Curve Plot ---
plt.figure()
plt.errorbar(nova_df['mjd'], nova_df['calibrated_mag'], yerr=nova_df['calibrated_mag_err'], fmt='bo-', capsize=4)
plt.xlabel("MJD")
plt.ylabel("Calibrated Magnitude (Sloan r')")
plt.title("Nova Light Curve")
plt.gca().invert_yaxis()
plt.grid(True)
plt.show()

# --- Save Nova Photometry Results to CSV ---
output_file = os.path.join(wd, 'nova_photometry_results.csv')
nova_df.to_csv(output_file, index=False)
print(f"Nova photometry results saved to {output_file}")

FileNotFoundError: [Errno 2] No such file or directory: 'C:\\Users\\friesco\\workstation\\fr-p\\studies\\ASTRO716\\data_excercise\\calibration_results.csv'